# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

### 更多BeautifulSoup語法應用: https://blog.csdn.net/winterto1990/article/details/47624167

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import os
import sys

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "文豪野犬"  # 這裡可以自己定義有興趣的關鍵字
utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元 repr和str用法差不多
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果
# 網路爬蟲
# https://zh.wikipedia.org/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 這個就是網址，可以比對看看編碼結果
# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

文豪野犬: %E6%96%87%E8%B1%AA%E9%87%8E%E7%8A%AC
/wiki/%E6%96%87%E8%B1%AA%E9%87%8E%E7%8A%AC


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [3]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
# 整個內文的div
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text()) # 這邊也可以用.text，不會報錯的

《文豪Stray Dogs》（日語：文豪ストレイドッグス，又譯：文豪野犬），是由朝霧卡夫卡（日語：朝霧カフカ）擔當原作、春河35作畫的日本漫畫作品。本作故事是以世界上的知名文豪或其作品為原型，使用異能力戰鬥的故事。

於2015年8月宣布決定製作電視動畫，[1]第1季於2016年4月至6月播出，第2季於2016年10月至12月播出，第3季於2019年4月至6月播出。此外，於2018年3月3日上映電影版。

被孤兒院趕出的少年中島敦恰巧撞見正在鶴見川（日語：鶴見川）「入水自殺」的武裝異能偵探社社員太宰治並把他救出、再一同解決「食人虎事件」後被推薦加入武裝異能偵探社。隨後陸續有港口黑幫成員試圖綁架敦，原來敦在國外異能組織「組合」的要求下被懸賞70億。

於2015年8月宣布決定製作電視動畫，第1季於2016年4月至6月播出，第2季於2016年10月至12月播出，第3季於2019年4月至6月播出。

於2017年6月19日宣布決定製作電影版《文豪Stray Dogs DEAD APPLE》，於2018年3月3日在日本上映，台灣於2018年5月4日上映，由曼迪代理。

2017年2月19日，與見面會『迷ヰ 犬達ノ宴 其ノ弐』發表舞台劇計劃，並預訂2017年12月在橫濱，2018年2月在東京上演。

文豪ストレイドッグス DEAD APPLE 獲 劇場版作品第一名

動畫第三季 獲 TV動畫 第二名



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [4]:
for ext_link in content:
#     \S :不是white space，(?!....): 之後的字符串內容需要不匹配才算匹配成功
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$")) # 利用RE
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [春河35] /wiki/%E6%98%A5%E6%B2%B335
外部連結: [日本漫畫] /wiki/%E6%97%A5%E6%9C%AC%E6%BC%AB%E7%95%AB
外部連結: [電視動畫] /wiki/%E9%9B%BB%E8%A6%96%E5%8B%95%E7%95%AB
外部連結: [孤兒院] /wiki/%E5%AD%A4%E5%85%92%E9%99%A2
外部連結: [中島敦] /wiki/%E4%B8%AD%E5%B3%B6%E6%95%A6
外部連結: [入水] /wiki/%E5%85%A5%E6%B0%B4
外部連結: [太宰治] /wiki/%E5%A4%AA%E5%AE%B0%E6%B2%BB


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存成檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [5]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        output_file = './/Data//WikiArticle//' + key_word.replace('?', '_') +'.txt' 
        with open(output_file, 'w', encoding = 'utf-8') as fh:
            for paragraph in content:
                fh.write(paragraph.get_text())
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href = re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       
                    a_keyword = link_string.get_text()
                    external_link_dict[a_link] = a_keyword
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [6]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2
WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E6%96%87%E8%B1%AA%E9%87%8E%E7%8A%AC (文豪野犬)
遞迴層[1] - /wiki/%E6%98%A5%E6%B2%B335 (春河35)
遞迴層[2] - /wiki/%E6%97%A5%E6%9C%AC (日本)
遞迴層[2] - /wiki/%E6%BC%AB%E7%95%AB%E5%AE%B6 (漫畫家)
遞迴層[2] - /wiki/%E6%8F%92%E7%95%AB%E5%AE%B6 (插畫家)
遞迴層[2] - /wiki/%E7%A5%9E%E5%A5%88%E5%B7%9D%E7%B8%A3 (神奈川縣)
遞迴層[2] - /wiki/%E6%A9%AB%E6%BF%B1%E5%B8%82 (橫濱市)
遞迴層[2] - /wiki/%E6%96%87%E8%B1%AAStray_Dogs (文豪Stray Dogs)
遞迴層[1] - /wiki/%E6%97%A5%E6%9C%AC%E6%BC%AB%E7%95%AB (日本漫畫)
遞迴層[2] - /wiki/File:Ja-Manga.oga ()
遞迴層[2] - /wiki/%E6%97%A5%E8%AA%9E%E6%9B%B8%E5%AF%AB%E7%B3%BB%E7%B5%B1 (?)
遞迴層[2] - /wiki/%E6%97%A5%E6%9C%AC (日本)
遞迴層[2] - /wiki/%E5%8D%A1%E9%80%9A (卡通)
遞迴層[2] - /wiki/%E6%BC%AB%E7%95%AB (漫畫)
遞迴層[2] - /wiki/%E6%BC%AB%E7%95%AB%E9%9B%9C%E8%AA%8C (漫畫雜誌)
遞迴層[2] - /wiki/%E5%8D%95%E8%A1%8C%E6%9C%AC (單行本)
遞迴層[2] - /wiki/File:Ja-komikkusu.oga ()
遞迴層[2] - /wiki/Project:%E4%B8%8A%E4%BC%A0?wpDestFile=Ja-komikkusu.oga (漫画本)
遞迴層[2] - /wiki/%E8%8A%92%E6%9E%9C (芒果)
遞迴層[2] - /wiki/%E6%97%A5%E5%9C%93 (日圓)
遞迴層[2]

遞迴層[2] - /wiki/%E5%8D%8A%E8%A7%84%E7%AE%A1 (半規管)
遞迴層[2] - /wiki/%E5%B9%B3%E8%A1%A1 (平衡)
遞迴層[2] - /wiki/Wikipedia:%E5%88%97%E6%98%8E%E6%9D%A5%E6%BA%90 ([來源請求])
遞迴層[2] - /wiki/%E5%B1%8D%E9%AB%94 (屍體)
遞迴層[2] - /wiki/%E8%85%90%E7%88%9B (腐爛)
遞迴層[2] - /wiki/%E6%B0%A3%E9%AB%94 (氣體)
遞迴層[2] - /wiki/%E8%AD%A6%E5%AF%9F (警察)
遞迴層[2] - /wiki/%E5%B7%A8%E4%BA%BA (巨人)
遞迴層[2] - /wiki/%E8%A6%AA%E5%B1%AC (親屬)
遞迴層[1] - /wiki/%E5%A4%AA%E5%AE%B0%E6%B2%BB (太宰治)
遞迴層[2] - /wiki/%E9%9D%92%E6%A3%AE%E7%B8%A3 (青森縣)
遞迴層[2] - /wiki/%E7%84%A1%E8%B3%B4%E6%B4%BE (無賴派)
遞迴層[2] - /wiki/%E5%B0%8F%E8%AA%AA%E5%AE%B6 (小說家)
遞迴層[2] - /wiki/%E9%8A%80%E5%BA%A7 (銀座)
遞迴層[2] - /wiki/%E8%8A%A5%E5%B7%9D%E8%B3%9E (芥川賞)
遞迴層[2] - /wiki/%E6%96%9C%E9%99%BD_(%E5%B0%8F%E8%AA%AA) (斜陽)
遞迴層[2] - /wiki/%E5%B1%B1%E5%B4%8E%E5%AF%8C%E6%A6%AE (山崎富榮)
遞迴層[2] - /wiki/%E7%8E%89%E5%B7%9D%E4%B8%8A%E6%B0%B4 (玉川上水)
遞迴層[2] - /wiki/%E4%BA%BA%E9%96%93%E5%A4%B1%E6%A0%BC (人間失格)
遞迴層[2] - /wiki/%E8%A1%80%E5%9E%8B (血型)
遞迴層[2] - /wiki/%E5%B0%8D%E9%A6%AC%E5%9C%8B (對馬國